In [7]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
import pandas as pd
import datetime
import import_ipynb
import utils_cdt as cdt

In [8]:
infoDataBases = {"ip":"","base":""}

In [9]:
def cache(self): 
    """ 
    Persist this RDD with the default storage level (C{MEMORY_ONLY_SER}). 
    """ 
    self.is_cached = True 
    self.persist(StorageLevel.MEMORY_ONLY_SER) 
    return self 

In [10]:
spark = SparkSession \
    .builder \
    .appName("Modelo - Spark") \
    .config("spark.executor.memory", "4gb") \
    .config("spark.driver.host","127.0.0.1") \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Modelo - Spark>

In [11]:
consulta = cdt.resultQuery("creditos_suspeitos.sql")

In [ ]:
df_daily = cdt.extrairDados(infoDataBases,consulta,"../data/","creditossuspeitos",spark=True,objSpark=spark)

In [ ]:
#Get all distinct Id_Conta
df_ids = df.groupBy('Id_Conta').count().toPandas()

In [ ]:
from pyspark.sql.functions import lit
sc = spark.sparkContext
sqlContext = SQLContext(spark)

#Create an empty dataframe to append de results
df_teste = sqlContext.createDataFrame(sc.emptyRDD(), df_daily.schema)
df_teste = df_teste.withColumn('ZScore',lit(None))

In [ ]:
#Iterate over the id's to calculate the mean and the standard deviation from each account and then calculate its Z-Score
#create sql query    
consulta_historica = cdt.resultQuery("mdl_creditos_suspeitos.sql")

#connect to the database anda extract register 3 months before the transaction from the specified Id_Conta
df_historical = cdt.extrairDados(infoDataBases,consulta_historica,spark=True,objSpark=spark)

In [ ]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
import time
start_time = time.time()

for index, row in df_ids.iterrows():
    #df_historical_id = df_historical.where('Id_Conta = '+ str(row['Id_Conta']))

    #Get the Mean and Standard Deviation from his historical data
    df_stats = df_historical.where('Id_Conta = '+ str(row['Id_Conta'])).select(
        _mean(col('Valor')).alias('mean'),
        _stddev(col('Valor')).alias('std'),
    ).collect()

    mean = df_stats[0]['mean']
    std = df_stats[0]['std']
    
    #Calculate the Z-Score of that account
    #df_out = df_spark.where('Id_Conta = '+str(row['Id_Conta'])).withColumn('ZScore', (df_spark.Valor - mean)/std)
    
    #Append to the final DataFrame
    df_teste = df_teste.union(df_daily.where('Id_Conta = '+str(row['Id_Conta'])).withColumn('ZScore', (df_daily.Valor - mean)/std))
    
#Calculate processing time
print(str(time.time() - start_time))

In [ ]:
outliers = df_teste.where('Zscore > 3').toPandas()
outliers.to_csv('outliersPortadores.csv')